## Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
sns.set_style('dark')

from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import roc_auc_score
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.svm import SVC

from tqdm import tqdm_notebook as tqdm
import tensorflow as tf

## Load the data
We set a few flags for whether we are running locally or on Kaggle, or whether we want the reduced or full dataset. For the reduced dataset, the test set consists of samples from the full train set. For use during model evaluation, this has been augmented with reverse-engineered targets. This is simply to allow us to evaluate the AUC score on this set as well - it is never used for training.

In [2]:
IS_LOCAL = True  # Sets whether we are running locally or on kaggle
USE_REDUCED = True  # Sets whether we should use the smaller dataset
data_index = 2*int(IS_LOCAL) + int(USE_REDUCED)
train_path = ('../input/santander-customer-transaction-prediction/train.csv',
             '../input/santandersmall/train_small.csv',
             'train.csv',
             'train_small.csv')[data_index]
test_path = ('../input/santander-customer-transaction-prediction/test.csv',
             '../input/santandersmall/test_small_with_targets.csv',
             'test.csv',
             'test_small.csv')[data_index]

train_df = pd.read_csv(train_path)
test_df = pd.read_csv(test_path)

In [3]:
features = [col for col in train_df.columns if col not in ['target', 'ID_code']]
if not 'target' in test_df:
    test_df['target'] = -1

all_df = pd.concat([train_df, test_df], sort=False)

## Feature Engineering

### Removing fake test samples
It was discovered during the competition that some of the test samples were synthetic. Moreover, it was stated that not all of the data in the test set was used for evaluation, and the synthetic data corresponds to this. In order to achieve the best scores possible, the fake samples should be removed before calculating features such as counts. We calculate the indices of the fake rows below using the method provided by the Kaggle user YaG320. When using the reduced dataset, we have no synthetic samples. This is handled correctly by the algorithm below.

In [4]:
unique_count = np.zeros((test_df.shape[0], len(features)))

for f, feature in tqdm(enumerate(features), total=len(features)):
    _, i, c = np.unique(test_df[feature], return_counts=True, return_index=True)
    unique_count[i[c == 1], f] += 1

real_sample_indices = np.argwhere(np.sum(unique_count, axis=1) > 0)[:, 0]
synthetic_sample_indices = np.argwhere(np.sum(unique_count, axis=1) == 0)[:, 0]
print('Real:', len(real_sample_indices))
print('Synthetic:', len(synthetic_sample_indices))

del unique_count


Real: 70065
Synthetic: 0


### Calculate counts
Counts of the values of the different features is often a powerful feature in itself. We calculate these below.

In [5]:
all_real_df = pd.concat([train_df, test_df.iloc[real_sample_indices, :]], sort=False)

for feature in tqdm(features):
    real_series = all_real_df[feature]
    
    # We only use the real samples to produce the count
    counts = real_series.groupby(real_series).count()
    
    full_series = all_df[feature]
    all_df[f'{feature}_count'] = full_series.map(counts)

del all_real_df
del real_series
del full_series

### Statistical Properties
Some statistical properties of the rows of data can also help the classifier make correct predictions by revealing connections that may otherwise be difficult to discover.

In [6]:
all_df['sum'] = all_df[features].sum(axis=1)
all_df['mean'] = all_df[features].mean(axis=1)
all_df['min'] = all_df[features].min(axis=1)
all_df['max'] = all_df[features].max(axis=1)
all_df['std'] = all_df[features].std(axis=1)
all_df['median'] = all_df[features].median(axis=1)
all_df['skew'] = all_df[features].skew(axis=1)
all_df['kurt'] = all_df[features].kurt(axis=1)

statistical_features = ['mean', 'min', 'max', 'std', 'median', 'skew', 'kurt']
# Due to normalization, mean and sum become the same value, so we only include one of them

### Normalization
Normalizing features to have zero mean, unit variance can often speed up training.

In [7]:
for feature in tqdm(features + statistical_features):
    if feature in features:
        all_df[feature] = StandardScaler().fit_transform(all_df[feature].values.reshape(-1, 1))
        all_df[f'{feature}_count'] = MinMaxScaler().fit_transform(all_df[f'{feature}_count'].values.reshape(-1, 1))
    if feature in statistical_features:
        all_df[feature] = MinMaxScaler().fit_transform(all_df[feature].values.reshape(-1, 1))

### Update feature list
We update the feature list with our new features, so that they are included in training and prediction.

In [8]:
for f in range(len(features)):
    features.append(f'{features[f]}_count')
features.extend(statistical_features)

### Splitting datasets back up
We are now done with our feature engineering, so we can split the data back into train and test sets.

In [9]:
train_df = all_df.iloc[:train_df.shape[0], :]
test_df = all_df.iloc[train_df.shape[0]:, :]

del all_df

## Model Selection

### Neural Network

We use a neural network model to make predictions. 5-fold stratified cross validation is used, and we rely on early stopping, setting a large default number of epochs of 100. Binary cross-entropy is used for the loss function, which is a good fit for a binary classification task such as this one. The output of the network is a single output from a sigmoid function, which can be interpreted as a probability.

In [ ]:
N_SPLITS = 5
BATCH_SIZE = 256
EPOCHS = 100
EARLY_STOPPING_PATIENCE = 8

OPTIMIZER = tf.keras.optimizers.Nadam()
LOSS='binary_crossentropy'
METRICS=[tf.keras.metrics.AUC()]

In [ ]:
# def get_cnn_model():
#     model = tf.keras.models.Sequential([
#         tf.keras.layers.Reshape((len(features) * 1, 1), input_shape=(len(features) * 1,)),
#         tf.keras.layers.Dense(64, activation='relu'),
#         #tf.keras.layers.Conv1D(64, 1, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         #tf.keras.layers.Dense(128, activation='relu'),
#         #tf.keras.layers.Conv1D(128, 1, activation='relu'),
#         #tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(256, activation='relu'),
#         #tf.keras.layers.Conv1D(256, 1, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         #tf.keras.layers.Dense(512, activation='relu'),
#         #tf.keras.layers.Conv1D(512, 1, activation='relu'),
#         #tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Dense(1024, activation='relu'),
#         #tf.keras.layers.Conv1D(1024, 1, activation='relu'),
#         tf.keras.layers.BatchNormalization(),
#         #tf.keras.layers.Dense(2048, activation='relu'),
#         #tf.keras.layers.Conv1D(2048, 1, activation='relu'),
#         #tf.keras.layers.BatchNormalization(),
#         tf.keras.layers.Flatten(),
#         tf.keras.layers.Dense(1, activation='sigmoid'),
#     ])
    
#     return model
def get_cnn_model():
    model = tf.keras.models.Sequential([
        tf.keras.layers.Reshape((len(features) * 1, 1), input_shape=(len(features) * 1,)),
        tf.keras.layers.Conv1D(32, 1, activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Conv1D(64, 1, activation='elu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(1024, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.005)),
        #tf.keras.layers.Dropout(0.4),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(512, activation='relu', kernel_regularizer=tf.keras.regularizers.l2(0.001)),
        #tf.keras.layers.Dropout(0.3),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(256, activation='relu'),
        #tf.keras.layers.Dropout(0.1),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.BatchNormalization(),
        tf.keras.layers.Dense(1, activation='sigmoid'),
    ])
    
    return model

In [ ]:
kfold = StratifiedKFold(n_splits=N_SPLITS, shuffle=True, random_state=42)

# Due to memory issues with saving all models, we instead make the predictions as each fold is trained.
#models = []
histories = []
oof_preds_sum = np.zeros((train_df.shape[0],))
train_preds_sum = np.zeros((train_df.shape[0],))
test_preds_sum = np.zeros((test_df.shape[0],))

for fold_num, (train_index, val_index) in tqdm(enumerate(kfold.split(train_df[features].values, train_df['target'].values)), total=N_SPLITS):
    print(f'Fold {fold_num+1}/{N_SPLITS}:')
    
    X_train = train_df.loc[train_index, features].values
    y_train = train_df.loc[train_index, 'target'].values.reshape(-1, 1)
    X_val = train_df.loc[val_index, features].values
    y_val = train_df.loc[val_index, 'target'].values.reshape(-1, 1)
    
    model = get_cnn_model()
    model.compile(optimizer=OPTIMIZER, loss=LOSS, metrics=METRICS)
    
    early_stopping_callback = tf.keras.callbacks.EarlyStopping(monitor='val_auc', mode='max', patience=EARLY_STOPPING_PATIENCE, restore_best_weights=True)
    
    history = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=EPOCHS, batch_size=BATCH_SIZE, callbacks=[early_stopping_callback])
    histories.append(history)
    
    
    print(f'Creating predictions for fold {fold_num + 1}/{N_SPLITS}')
    val_preds = model.predict(X_val)
    train_preds = model.predict(X_train)
    test_preds = model.predict(test_df[features].values)
    
    oof_preds_sum[val_index] += val_preds[:, 0]
    train_preds_sum[train_index] += train_preds[:, 0]
    test_preds_sum += test_preds[:, 0]
    
    val_auc = roc_auc_score(y_val, val_preds)
    print(f'Fold validation AUC: {val_auc}')
    print()
    #models.append(model)

In [ ]:
train_preds_sum.shape

### Training plots

In [ ]:
plt.figure(figsize=(20, 5 * (len(histories) // 5 + 1)))
for i, h in enumerate(histories):
    plt.subplot(len(histories) // 5 + 1, 5, i+1)
    plt.plot(h.history['loss'], label='Train loss')
    plt.plot(h.history['val_loss'], label='Val loss')
    plt.plot(h.history['auc'], label='Train AUC')
    plt.plot(h.history['val_auc'], label='Val AUC')
    plt.legend()

## Evaluate performance

In [ ]:
# train_preds = np.zeros(train_df.shape[0])
# test_preds = np.zeros(test_df.shape[0])

# for model in models:
#     pred_train = model.predict(train_df[features].values)
#     pred_test = model.predict(test_df[features].values)
    
#     train_preds += pred_train
#     test_preds += pred_test

# train_preds /= len(models)
# test_preds /= len(models)

In [ ]:
#train_preds = train_preds[:, 0]
#test_preds = test_preds[:, 0]

In [ ]:
train_preds_sum /= N_SPLITS
oof_preds_sum /= N_SPLITS
test_preds_sum /= N_SPLITS

In [ ]:
train_auc = roc_auc_score(train_df['target'], train_preds_sum)
oof_auc = roc_auc_score(train_df['target'], oof_preds_sum)
print(f'Train AUC: {train_auc}')
print(f'Out of fold AUC: {oof_auc}')

We attempt to load the small test set with targets (reverse engineered from train set), which allows us to evaluate test set performance.

In [ ]:
try:
    test_df = pd.read_csv('test_small_with_targets.csv')
except FileNotFoundError:
    pass

In [ ]:
if test_df['target'][0] != -1:
    test_auc = roc_auc_score(test_df['target'], test_preds_sum)
    print(f'Test AUC: {test_auc}')

### Create submission

In [ ]:
sub = pd.DataFrame({'ID_code': test_df['ID_code'], 'target': test_preds_sum})
sub.to_csv('submission.csv', index=False)

In [ ]:
from IPython.display import FileLink
FileLink('submission.csv')